# Imports

In [1]:
import torch
!pip install torch-geometric torch-scatter torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install ogb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.13.1+cu116.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.0/565.0 KB 12.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 35.5 MB/s eta 0:00:00
  Created wheel for torch-geometric: filename=torch_geometric-2.2.0-py3-none-any.whl size=773302 sha256=f39de97d2ad678c60a8e2935d8b0dad4bb93caa725745fa550c50798219d1662
  Stored in directory: /root/.cache/pip/wheels/31/b2/8c/9b4bb72a4384eabd1ffeab2b7ead692c9165e35711f8a9dc72
Successfully built torch-geometric
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 KB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for li

In [2]:
import multiprocessing as mp
import os
import pickle
import warnings
import torch
from functools import partial
from itertools import combinations
import numpy as np
from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import cosine_similarity as cos_sim
from sklearn.metrics.pairwise import linear_kernel
from sklearn.neighbors import BallTree
from sklearn.preprocessing import StandardScaler, normalize
from tqdm import tqdm
from ogb.nodeproppred import PygNodePropPredDataset

from scipy.linalg import orthogonal_procrustes
from torch_geometric.datasets import Planetoid, Coauthor

In [3]:
# use google drive for saving and loading information
from google.colab import drive
import pickle
import os

drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/L45_project/'

Mounted at /content/drive


# Cora Data Set Stats

In [4]:
cora_dataset = Planetoid("/tmp/cora", name="cora", split="full")
cora_data = cora_dataset[0]

Processing...
Done!


In [5]:
# Create a random list of indices with 10 items from each class for 2nd cossim
cora_labels = cora_dataset[0].y.detach().numpy()
cora_num_classes = 7
num_samples = 10
cora_indices = []
for i in range(cora_num_classes):
  class_i = np.random.choice(np.where(cora_labels == i)[0], size=num_samples, replace=False)
  print(class_i)
  cora_indices += class_i.tolist()


[2576 1912 2200  423 2116 1296 2219 2266 1203  751]
[ 589 1767  655 2478 1009  112  910  876 2144 2532]
[ 860 1599 1718  823  564  867  353 1444 1358 1037]
[2610 2204  674 1186 1692 2241 1865 2668  386 1175]
[ 638 1484 1001 2223 2357 2670 1234 2473  498   63]
[2553 1953 1825 2103 1578 1394  941 1979 2275  793]
[2110  939 2157  778  213  123  331 1170 2608  908]


# CoauthorCS Data Set Stats

In [74]:
cs_dataset = Coauthor("/tmp/coauthor", name="CS")
cs_data = cs_dataset[0]
cs_data

Processing...
Done!


Data(x=[18333, 6805], edge_index=[2, 163788], y=[18333])

In [75]:
# Create a random list of indices with 10 items from each class for 2nd cossim
cs_labels = cs_dataset[0].y.detach().numpy()
cs_num_classes = 15
num_samples = 10
cs_indices = []
for i in range(cs_num_classes):
  class_i = np.random.choice(np.where(cs_labels == i)[0], size=num_samples, replace=False)
  print(class_i)
  cs_indices += class_i.tolist()


[ 1738 11908   164  5685  4882  2188 15054 14202  6519 11263]
[17340  2739  7216  6967  2434  9553   361  3532  1727  6344]
[ 6746  5672  2103  6218 13375  2596  8004 12211 17863 14256]
[ 4861  2871  9022  2756 17701 11697 17024 14378  8596  6382]
[14787 17693 16931  3684  2107  4098  4252 18079 12822 14164]
[12010  9075 14639  2645 15214 10014  5268  6928  1169  8939]
[13148 14500 14525  8582   988 16381  1460 15069   530  3596]
[16870 12410  6654 15057 10901  7034  5804 12884 12318 14082]
[14855 14133  8575 13112  8658 17572 10423 10580 13123  9389]
[ 5980  4926  3965   326   243 14402  1465 13669  8111  6104]
[17456  5971  1211 12764  2552 14878  2059 15918 10887  2982]
[ 4040 13890  2597  1052   270  6304  7733  3144 16988  9746]
[10056  9629  8290 12833  9149 14268  2792  9888  1479 16629]
[16784 16007 17898  7822 12449   572 14636  4305 11708   964]
[10477 16354 10186  6711  6782   715  8620 10329 13076 13599]


# Arxiv Data Set Stats

In [ ]:
d_name = "ogbn-arxiv"

arxiv_dataset = PygNodePropPredDataset(name = d_name)

split_idx = arxiv_dataset.get_idx_split()
train_idx, valid_idx, test_idx = split_idx["train"], split_idx["valid"], split_idx["test"]
arxiv_data = arxiv_dataset[0]
arxiv_data.y = arxiv_data.y.squeeze()
arxiv_data.node_year = arxiv_data.node_year.squeeze()
arxiv_data

Downloaded 0.08 GB: 100%|██████████| 81/81 [00:01<00:00, 43.28it/s]


Extracting dataset/arxiv.zip


Processing...


Loading necessary files...
This might take a while.
Processing graphs...


100%|██████████| 1/1 [00:00<00:00, 1737.49it/s]


Converting graphs into PyG objects...


100%|██████████| 1/1 [00:00<00:00, 4072.14it/s]

Saving...



Done!


Data(num_nodes=169343, edge_index=[2, 1166243], x=[169343, 128], node_year=[169343], y=[169343])

In [ ]:
# Create a random list of indices with 10 items from each class for 2nd cossim
arxiv_labels = arxiv_dataset[0].y.detach().numpy()
arxiv_num_classes = 15
num_samples = 10
arxiv_indices = []
for i in range(arxiv_num_classes):
  class_i = np.random.choice(np.where(arxiv_labels == i)[0], size=num_samples, replace=False)
  print(class_i)
  cs_indices += class_i.tolist()


[ 39838  83452 121005  72065 120836  33614 104457  92137 166835  89449]
[163199  50487  66258  20164  55813 141377 164081 152616 131507   7907]
[ 82068  73931 110838 157702 106276  55810 111240  52022 115509  93760]
[129633  27137  85147  93966  26410  88265  54493 124833 129571   5939]
[120595 165436  50616 111979 141603  80235  23864 155495 167339  43562]
[147926  96304 168765 116695  29632  52958  42149 103225 150136  18575]
[ 18358  26269 119274 164042  31333  96623 114025 163142 166729   6698]
[ 46360 157799  96427 165400  11855  48412 114634 169026 131173   2824]
[  4080  97610 103879  59235 153709  64641  69881 137656  41905 149905]
[113501  54510  26145 116171 120055  13141  42119 131513 123927 135053]
[ 54921  83081  29244 102887  48373  11656  42716   8950 156374  67367]
[134831  35357  50249  98685  17244 126726  53801  36857 104449  51917]
[162411  28709  28409  65874 126627  47486  53177 145824  92632  14455]
[112676  23049 157197  10517  65275  56907  23397 147847 137612 

# Metric Calculations

In [6]:
# Adapted from https://github.com/SGDE2020/embedding_stability/blob/master/lib/tools/comparison.py

class Comparison:

    def __init__(self, emb_dir, embeddings, num_nodes, file_prefix):
        """
        emb_dir: str, path where embeddings are saved
        embeddings: list, list of strings that specify the embedding files
        """
        self.dir = emb_dir
        self.embeddings = embeddings
        self.pairs = self._combinations(embeddings)
        self.num_vertices = num_nodes

        # use file names without numbering to mark result files
        self.file_prefix = file_prefix

    def _combinations(self, emb_list):
        """ Computes all different comparison pairs of a list of embeddings """
        return [pair for pair in combinations(emb_list, 2)]

    def _analyse_jaccard(self, queries, nodes, k, pair):
        """This function is called in the multiprocessing of jaccard score"""
        indices_0 = np.asarray(queries[pair[0]])
        indices_1 = np.asarray(queries[pair[1]])
        nodes = np.asarray(nodes)
        jaccard_score = {}
        for i in range(len(nodes)):
            jaccard_score[nodes[i]] = \
                len(np.intersect1d(indices_0[i, 1:(k + 1)], indices_1[i, 1:(k + 1)], assume_unique=True)) / \
                len(np.union1d(indices_0[i, 1:(k + 1)], indices_1[i, 1:(k + 1)]))
        return list(jaccard_score.values())

    def _analyse_second_cossim(self, queries, normed_embs, nodes, k, pair):
        """
        This function is called in the multiprocessing of the second order cosine similarity.
        """

        # Convert the indices of nearest neighbors back into numpy
        indices_0 = np.asarray(queries[pair[0]])
        indices_1 = np.asarray(queries[pair[1]])

        # Convert the embeddings and nodes back into numpy
        norm_emb_0 = np.asarray(normed_embs[pair[0]])
        norm_emb_1 = np.asarray(normed_embs[pair[1]])
        nodes = np.asarray(nodes)

        # Compute the second order cosine similarity
        pair_results = []
        for i in range(len(nodes)):
            # Build the set of nearest neighbors w.r.t. both embeddings
            # Use indices from 1 to k+1, because the first entry will always be the node itself
            neighbors_union = np.union1d(indices_0[i, 1:(k + 1)], indices_1[i, 1:(k + 1)])

            # Vectors of cosine similarity values of nearest neighbors. There was an error in the original source code, did not use norm_emb_1 for m1
            m0 = cos_sim(norm_emb_0[neighbors_union], norm_emb_0[nodes[i]].reshape(1, -1))
            m1 = cos_sim(norm_emb_1[neighbors_union], norm_emb_1[nodes[i]].reshape(1, -1))

            # Flatten output matrix
            assert m0.shape[1] == 1 and m1.shape[1] == 1, "m0 and m1 should only have a single variable in the second dimension"
            m0 = m0.flatten()
            m1 = m1.flatten()

            # Cosine similarity between similarity vectors
            pair_results.append(float(1-cosine(m0,m1)))
        return pair_results

    def k_nearest_neighbors(self, nodes=None, append=False, samples=100, k=10, jaccard=False, load=False,
                            kload_size=100, save=True, save_path=None, num_processes=4):
        """
        Computes the k nearest neighbors to some specified nodes with respect to cosine similarity. As an intermediate
        step, it computes the 100 nearest neighbors and saves them to file. Based on these neighbors, the k-nn overlaps
        are computed.
        Args:
            nodes: list, that specifies the nodes
            append: bool, if nodes are specified, whether additional nodes will be sampled and added to nodes
            samples: int, number of nodes that will be sampled
            k: int, number of neighbors that will be used in the comparison
            jaccard: bool, whether jaccard score or overlap will be used as similarity measure
            load: bool, whether a file of computed neighbors will be used
            kload_size: Size of k in knn file to load
            save: bool, whether the results should be saved in a text file
            save_path: str, path where the file will be saved
            num_processes: number of random processes in parallelization
        Returns:
            dict, "nodes": array of nodes, "overlaps": array of overlaps, columns are values per node; or array of
            jaccard scores
        """
        # Handle the nodes input: Whether to sample nodes, use given nodes, or both.
        nodes = self._get_nodes(nodes, samples, append)

        # Load k nearest neighbors to save time? Else we will have to compute them first.
        if load:
            file_name = self.file_prefix + "_" + str(kload_size) + "nns.pickle"
            with open(os.path.join(save_path, file_name), "rb") as pickle_file:
                queries = pickle.load(pickle_file)
            assert list(queries.keys()) == self.embeddings, ("Keys of loaded queries do not match"
                                                             " with available embeddings")

        # Normalizing the embeddings to be able to use distance as a proxy for cosine similarity
        # BallTree from sklearn is used to compute the neighbors efficiently
        else:
            queries = self.nearest_neighbors_queries(nodes, k, save_path)
        # Store the naive overlaps for all pairs

        print(f"\n\n {queries.keys()} \n")

        # Use multiprocessing to speed up overlap computation.
        # Too much data is passed to the processes which makes it inefficient.
        # Possibly, it is faster to store the data as a file as an intermediate step.
        # only run if less than 100 neighbors are queried, as too large neighborhoods may cause memory issues when
        # distributing tasks
        if num_processes > 1 and k <= 100:
            with mp.Pool(num_processes) as p:
                # arguments passed in multiprocessing must be pickable
                p_queries = queries
                p_nodes = nodes.tolist()
                if jaccard:
                    multiprocess_func = partial(self._analyse_jaccard, p_queries, p_nodes, k)
                else:
                    multiprocess_func = partial(self._analyse_knn, p_queries, p_nodes, k)
                li_overlap = []
                for result in tqdm(p.imap(multiprocess_func, self.pairs), total=len(self.pairs)):
                  li_overlap.append(result)
        else:
            if jaccard:
                li_overlap = [self._analyse_jaccard(queries, nodes.tolist(), k, pair) for pair in self.pairs]
            else:
                li_overlap = [self._analyse_knn(queries, nodes.tolist(), k, pair) for pair in self.pairs]

        # Convert the result into numpy
        overlap = np.asarray(li_overlap)

        # Save the results
        if jaccard:
            nodes_suffix = "jaccard_nodes"
            scores_suffix = f"{k}nn_jaccard"
        else:
            nodes_suffix = "overlap_nodes"
            scores_suffix = f"{k}nn_overlap"
        if save is True:
            np.save(os.path.join(save_path, f"{self.file_prefix}_{nodes_suffix}"), nodes)
            np.save(os.path.join(save_path, f"{self.file_prefix}_{scores_suffix}"), overlap)
        return {"nodes": nodes, "overlaps": overlap}

    def jaccard_similarity(self, nodes=None, append=False, samples=100, k=10, load=False, kload_size=100, save=True,
                           save_path=None, num_processes=4):
        """
        Alias for k_nearest_neighbors with jaccard=True.
        See k_nearest_neighbors for detailed documentation.
        """
        return self.k_nearest_neighbors(nodes=nodes, append=append, samples=samples, k=k, jaccard=True, load=load,
                                        kload_size=kload_size, save=save, save_path=save_path,
                                        num_processes=num_processes)


    def second_order_cosine_similarity(self, nodes=None, append=False, num_samples=1000, k=10, load=True, save=False,
                                       save_path=None, num_processes=4):
        """ Computes second order cosine similarity.
        Args:
            nodes: list, that specifies the nodes
            append: bool, if nodes are specified, whether additional nodes will be sampled and added to nodes
            num_samples: int, number of nodes that will be sampled
            k: int, number of neighbors that will be used in the comparison
            load: bool, whether to load nearest neighbors from file
            save: bool, whether the results should be saved
            save_path: str, path where the file will be saved
            num_processes: number of random processes in parallelization
        Returns:
            nodes: numpy array of used nodes
            results: numpy array of similarity values of size (number of embedding pairs, number of nodes)
        """

        # Handle the nodes input: Whether to sample nodes, use given nodes, or both.
        nodes = self._get_nodes(nodes, num_samples, append)

        # Load required data: nearest neighbors, embeddings
        normed_embs = {}

        if load:
            # Load nearest neighbors from file
            file_name = self.file_prefix + "_" + str(k)+ "nns.pickle"
            with open(os.path.join(save_path, file_name), "rb") as pickle_file:
                queries = pickle.load(pickle_file)
            assert list(queries.keys()) == self.embeddings, ("Keys of loaded queries do not match with "
                                                             "available embeddings")
            for emb in tqdm(self.embeddings, desc="Loading nearest neighbor files"):
                normed_embs[emb] = normalize(self.read_embedding(os.path.join(self.dir, emb)), norm='l2', copy=False)
        else:
            queries, normed_embs = self.nearest_neighbors_queries(nodes, k, save_path, return_embeddings=True)

        # Start computation of second order cosine similarity
        # arguments passed in multiprocessing must be pickable
        p_normed_embs = dict([(key, norm_emb.tolist()) for key, norm_emb in normed_embs.items()])
        p_nodes = nodes.tolist()
        # Avoid multiprocessing on Colab, not sure if it works locally though
        if num_processes > 1 and k <= 100:
            with mp.Pool(num_processes) as p:
                li_results = p.map(partial(self._analyse_second_cossim, queries, p_normed_embs, p_nodes, k), self.pairs)
              # li_results = []
              # partial_func = partial(self._analyse_second_cossim, queries, p_normed_embs, p_nodes, k) 
              # for result in tqdm(p.imap(partial_func, self.pairs), total=len(self.pairs)):
              #   li_results.append(result)
        else:
          li_results = []
          for pair in tqdm(self.pairs, desc="Comparing embeddings"):
            li_results.append(self._analyse_second_cossim(queries, p_normed_embs, p_nodes, k, pair))

        results = np.asarray(li_results)

        if save is True:
            np.save(os.path.join(save_path, f"{self.file_prefix}_{k}nn_2nd_order_cossim"), results)

        return nodes, results

    def nearest_neighbors_queries(self, nodes, k, save_path, return_embeddings=False):
        """Uses a ball tree to compute the nearest neighbors in the embedding space"""
        queries = {}
        normed_embs = {}
        # Normalizing the embeddings to be able to use distance as a proxy for cosine similarity
        # BallTree from sklearn is used to compute the neighbors efficiently
        if return_embeddings:
            for emb in tqdm(self.embeddings, desc="Querying nearest neighbors"):
                normed_embs[emb] = normalize(self.read_embedding(os.path.join(self.dir, emb)), norm='l2', copy=False)
                ball_tree = BallTree(normed_embs[emb], leaf_size=40)
                queries[emb] = ball_tree.query(normed_embs[emb][nodes, :], k=k + 1, return_distance=False).tolist()
        else:
            for emb in tqdm(self.embeddings, desc="Querying nearest neighbors"):
                normalized_embedding = normalize(self.read_embedding(os.path.join(self.dir, emb)), norm='l2',
                                                 copy=False)
                ball_tree = BallTree(normalized_embedding, leaf_size=40)
                # Query the k+1 nearest neighbors, because a node will always be the closest neighbor to itself
                queries[emb] = ball_tree.query(normalized_embedding[nodes, :], k=k + 1, return_distance=False).tolist()

        # Save the computed neighbors to be able to skip the computation
        self.save_pickle(queries, save_path, self.file_prefix + "_" + str(k) + "nns")

        if return_embeddings:
            return queries, normed_embs
        else:
            return queries

    def sample_nodes(self, k):
        """
        Sample unique nodes of an embedding
        Args:
            k: int, number of nodes to sample
        Returns:
            numpy array of node ids of length k if k is smaller than the number of nodes available.
            Otherwise, all available nodes are returned.
        """
        vertices = np.arange(self.num_vertices)
        np.random.shuffle(vertices)
        return vertices[:min(k, self.num_vertices)]


    def cossim_analysis(self, save_path):
        """ Computes aligned cosine similarity values. Internally performs orthogonal transformation (Procrustes
        problem) between two embeddings and saves transformation matrices as well as vector of resulting errors
        """

        # Set up file naming
        results_suffix = "aligned_cossim"

        # Read the embeddings
        normed_embs = {}
        for emb in tqdm(self.embeddings, desc="Reading embeddings"):
            normed_embs[emb] = normalize(
                    self.read_embedding(os.path.join(self.dir, emb))[np.arange(self.num_vertices)], norm='l2',
                    copy=False)

        # Do the analysis
        emb_ind = -1
        results = []
        for pair in tqdm(self.pairs, desc="Comparing embeddings"):

            # only update first embedding if it does not change
            if emb_ind != pair[0]:
                emb_ind = pair[0]

            W1 = normed_embs[emb_ind]

            # transform W2 into W1 using procrustes matrix
            Q, _ = orthogonal_procrustes(normed_embs[pair[1]], normed_embs[pair[0]], check_finite=False)
            W2 = normed_embs[pair[1]].dot(Q)

            # Do 1-cosine to get the actual cosine similarity instead of cosine difference
            pair_results = np.array([1-cosine(W1[i], W2[i]) for i in range(self.num_vertices)])
            results.append(pair_results)

        results = np.asarray(results)
        np.save(os.path.join(save_path, f"{self.file_prefix}_{results_suffix}"), results)
        return np.arange(self.num_vertices), results

    def unaligned_cosine_analysis(self, save_path):
        """ Computes regular cosine similarity values, without alignment, between embedding pairs
        """

        # Set up file naming
        results_suffix = "cossim"

        # Read the embeddings
        normed_embs = {}
        for emb in tqdm(self.embeddings, desc="Reading embeddings"):
            normed_embs[emb] = normalize(
                    self.read_embedding(os.path.join(self.dir, emb))[np.arange(self.num_vertices)], norm='l2',
                    copy=False)

        # Do the analysis
        emb_ind = -1
        results = []
        for pair in tqdm(self.pairs, desc="Comparing embeddings"):

            # only update first embedding if it does not change
            if emb_ind != pair[0]:
                emb_ind = pair[0]

            # Get our two embedding matrics
            W1 = normed_embs[emb_ind]
            W2 = normed_embs[pair[1]]

            # Do 1-cosine to get the actual cosine similarity instead of cosine distance
            pair_results = np.array([1-cosine(W1[i], W2[i]) for i in range(self.num_vertices)])
            results.append(pair_results)

        results = np.asarray(results)
        np.save(os.path.join(save_path, f"{self.file_prefix}_{results_suffix}"), results)
        return np.arange(self.num_vertices), results

    def pairwise_distance(self, nodes, save_path, norm=False):
        """Calculate the pairwise distance between all our embeddings. Save these to the given file path"""

        results_suffix = "euclidean_distance"

        # Read the embeddings
        normed_embs = {}
        for emb in tqdm(self.embeddings, desc="Reading embeddings"):
            # Option for calculating distance between normalized embeddings
            if norm:
              normed_embs[emb] = normalize(
                      self.read_embedding(os.path.join(self.dir, emb))[np.arange(self.num_vertices)], norm='l2',
                      copy=False)
            else:
              normed_embs[emb] = self.read_embedding(os.path.join(self.dir, emb))[np.arange(self.num_vertices)]

        # Do the analysis
        emb_ind = -1
        results = []
        for pair in tqdm(self.pairs, desc="Comparing embeddings"):

            # only update first embedding if it does not change
            if emb_ind != pair[0]:
                emb_ind = pair[0]

            # Get the distance between the two matrices of embeddings and add to results array
            results.append(np.linalg.norm(normed_embs[emb_ind] - normed_embs[pair[1]], axis=1))

        results = np.asarray(results)
        np.save(os.path.join(save_path, f"{self.file_prefix}_{results_suffix}"), results)


    def _get_nodes(self, nodes, num_samples, append):
        """
        Handles getting nodes for the experiments.
        Args:
            nodes: list, node ids
            num_samples: int, how many nodes should be sampled
            append: bool, whether to append sampled nodes to specified nodes
        Returns:
            numpy array of (sampled) node ids
        """
        if nodes is None:
            nodes = self.sample_nodes(num_samples)
        elif append is True:
            # allows specified nodes to be taken twice
            nodes.extend(self.sample_nodes(num_samples))
        return np.asarray(nodes)

    def save_pickle(self, obj, save_path, file_name):
        if save_path is None:
            save_path = os.getcwd()
        if file_name is None:
            # generate name of report from embedding input: use name of first embedding file without number information
            file_name = self.file_prefix
        with open(os.path.join(save_path, f"{file_name}.pickle"), "wb") as f:
            pickle.dump(obj, f)

    def get_combinations(self):
        return self.pairs

    def get_vertex_count(self):
        return self.num_vertices

    # The original code reorders the nodes by ID, but our nodes have the same ordering so there is no need to reorder
    def read_embedding(self, path):
      node_embedding = torch.load(path, map_location=torch.device('cpu')).detach().numpy()
      return node_embedding

In [ ]:
# Code adapted from https://github.com/SGDE2020/embedding_stability/blob/master/similarity_tests/similarity_tests.py
TESTS= ["cossim", "jaccard", "2ndcos", "unalign_cossim", "dist"]

# Expects the the file names in the embedding directory to be of the form model-name_seed_emb.pt
def run_tests(embedding_dir, file_prefix, tests=TESTS, num_nodes=-1, knn_size=20, load_knn=False, kload_size=None,
                results_dir=None, num_processes=4, cossim_sec_indices=None):
    """
    Run specified similarity tests.
    Params:
        embedding_dir: str, path to directory where embeddings are stored
        file_prefix: the prefix for files for saving results
        tests: list of str, tests that will be conducted
        num_nodes: int, number of nodes of a graph that are used in the tests (just the number of nodes in the graph)
        knn_size: int, neighborhood size for knn test
        load_knn: bool, whether a stored knn matrix should be loaded
        nodeinfo_dir = str, path to directory where nodeinfo is stored (tables)
        results_dir = str, path to directory where results will be saved to
    """
    # Store results in the embedding directory in a results file if no other location is specified
    if results_dir is None:
      results_dir = embedding_dir + "results/"
    # Create results directory if it does not exist
    if not os.path.exists(results_dir):
        os.mkdir(results_dir) 

    # Computes a list of all _emb.pt files in a given directory. Assumes the directory being used only contains results from one architecture and data set
    fnames = sorted([f for f in os.listdir(embedding_dir) if f.endswith("_emb.pt")])
    print(fnames)

    if len(fnames) <= 1:
        print(f"Did not find any embeddings for in directory {embedding_dir}")
 
    else:
        # To be compatible with the original source code, we need a list of nodes indices we will be using in our metric computations
        # We use all the nodes
        nodes = [i for i in range(num_nodes)]

        # Allow us to use specific nodes for the second order cossine similarity as it can be very computational expensive
        cossim_nodes = nodes if cossim_sec_indices is None else cossim_sec_indices

        if load_knn and kload_size is None:
            kload_size = knn_size

        # Start tests
        comp = Comparison(emb_dir=embedding_dir, embeddings=fnames, num_nodes=num_nodes, file_prefix=file_prefix)
        if "cossim" in tests:
            print("Executing cosine similarity")
            comp.cossim_analysis(save_path=results_dir)
        if "jaccard" in tests:
            print("Executing jaccard score")
            comp.jaccard_similarity(
                nodes=nodes, append=False, k=knn_size,
                load=load_knn, kload_size=kload_size, save=True, save_path=results_dir, num_processes=num_processes
            )
        if "2ndcos" in tests:
            print("Executing second order cosine similarity")
            comp.second_order_cosine_similarity(
                nodes=cossim_nodes, append=False, k=knn_size,
                save=True, save_path=results_dir, num_processes=num_processes, load=load_knn
            )
        if "unalign_cossim" in tests:
            print("Executing cosine similarity")
            comp.unaligned_cosine_analysis(save_path=results_dir)
        if "dist" in tests:
            print("Executing euclidean distance")
            comp.pairwise_distance(nodes=nodes, save_path=results_dir)



# Run metric calculations

In [ ]:
NUM_NODES_CORA = cora_data.x.shape[0]
# CHANGE these to match you directory naming structure
model_directories = ["GIN_Cora_Layers_10", "GIN_Cora_Layers_5", "GAT_Cora_Layers_10", "GAT_Cora_Layers_5", "GraphSAGE_Cora_Layers_10", "GraphSAGE_Cora_Layers_5"]
for model_dir in model_directories:
  model_directory = model_dir
  model_file_prefix = model_directory+"_analysis"
  # model_directory = "GraphSAGE/"
  # model_file_prefix = "GraphSAGE_analysis"
  # Run the tests for tests=["cossim", "jaccard"], multiple processes work for these
  run_tests(embedding_dir=file_path+model_directory, file_prefix=model_file_prefix, tests=["cossim", "jaccard", "unalign_cossim", "dist"], num_nodes=NUM_NODES_CORA, num_processes=16)
  # Run the tests for tests=["2ndcos"], multiple processes does not work for this so only run on 1, but we can load the nearest neighbours from the previous metric calculations
  run_tests(embedding_dir=file_path+model_directory, file_prefix=model_file_prefix, tests=["2ndcos"], num_nodes=NUM_NODES_CORA, num_processes=1, load_knn=True)

['GIN_Cora_Layers_10_1024258131_emb.pt', 'GIN_Cora_Layers_10_116846604_emb.pt', 'GIN_Cora_Layers_10_1221215631_emb.pt', 'GIN_Cora_Layers_10_1517964140_emb.pt', 'GIN_Cora_Layers_10_170173784_emb.pt', 'GIN_Cora_Layers_10_1818027900_emb.pt', 'GIN_Cora_Layers_10_1860537279_emb.pt', 'GIN_Cora_Layers_10_1863727779_emb.pt', 'GIN_Cora_Layers_10_2036056847_emb.pt', 'GIN_Cora_Layers_10_2105922959_emb.pt', 'GIN_Cora_Layers_10_2135956485_emb.pt', 'GIN_Cora_Layers_10_2342954646_emb.pt', 'GIN_Cora_Layers_10_2455059856_emb.pt', 'GIN_Cora_Layers_10_2739899259_emb.pt', 'GIN_Cora_Layers_10_2794978777_emb.pt', 'GIN_Cora_Layers_10_2952735006_emb.pt', 'GIN_Cora_Layers_10_3300171104_emb.pt', 'GIN_Cora_Layers_10_3303475786_emb.pt', 'GIN_Cora_Layers_10_361232447_emb.pt', 'GIN_Cora_Layers_10_3647665043_emb.pt', 'GIN_Cora_Layers_10_3692371949_emb.pt', 'GIN_Cora_Layers_10_3710910636_emb.pt', 'GIN_Cora_Layers_10_400225693_emb.pt', 'GIN_Cora_Layers_10_4083009686_emb.pt', 'GIN_Cora_Layers_10_4193977854_emb.pt', 'GI

Reading embeddings:  63%|██████▎   | 19/30 [00:07<00:04,  2.45it/s]


KeyboardInterrupt: ignored

# Compute metric averages

In [ ]:
def read_and_average_metric(file_suffix, class_labels, num_classes):
  results_arr = np.load(file_path+model_directory+"/results/"+model_file_prefix+"_"+file_suffix+".npy", mmap_mode='r')
  results_flat = results_arr.flatten() 
  # Compute the 5 number summary for the data across all classes, better than mean and std for potentially skewed distribution
  q1, q2, q3 = np.percentile(results_flat, [25,50,75])
  min, max = results_flat.min(), results_flat.max()
  # Print results
  print("TOTAL STATS")
  print("q1=" + str(q1) + ", q2=" + str(q2) + ", q3=" + str(q3) + ", min=" + str(min) + ", max=" + str(max))

  for i in range(num_classes):
    indices = np.where(class_labels==i)[0]
    # Filter the results by class
    class_vals = results_arr[:, indices]
    # Compute stats for the class
    q1_c, q2_c, q3_c = np.percentile(class_vals, [25,50,75])
    min_c, max_c = class_vals.min(), class_vals.max()
    # Print results
    print("CLASS STATS " + str(i))
    print("q1=" + str(q1_c), ", q2=" + str(q2_c) + ", q3=" + str(q3_c) + ", min=" + str(min_c) + ", max=" + str(max_c))


In [ ]:
# Read and compute aligned cosine metric
read_and_average_metric("aligned_cossim", cora_labels, cora_num_classes)

In [ ]:
# Read and compute jaccard metric
read_and_average_metric("20nn_jaccard", cora_labels, cora_num_classes)

In [ ]:
# Read and compute 2nd order cosine similarity metric
read_and_average_metric("20nn_2nd_order_cossim", cora_labels, cora_num_classes)

In [ ]:
# Read and compute 2nd order cosine similarity metric
read_and_average_metric("cossim", cora_labels, cora_num_classes)

In [ ]:
read_and_average_metric("euclidean_distance", cora_labels, cora_num_classes)

To do:
- Store the nodes being used if we're only using a subset
- Subsets for second_order_cosine_similarity for larger dataset

# prediction instability

In [53]:
import itertools
from collections import defaultdict
from pathlib import Path
import seaborn as sns
import matplotlib.figure
import matplotlib.pyplot as plt
from typing import List, Optional, Union
import logging
from torch_geometric.typing import Adj
import torch
import torch.nn as nn

log = logging.getLogger(__name__)

def max_pi(acc1: float, acc2: float) -> float:
    return min(1.0, 2.0 - acc1 - acc2)


def min_pi(acc1: float, acc2: float) -> float:
    return abs(acc1 - acc2)

def savefig(
    fig: Union[matplotlib.figure.Figure, sns.FacetGrid],
    path: Union[str, Path],
    fname: Optional[str] = None,
):
    if os.path.isdir(path):
        if fname is None:
            raise ValueError(
                "Path to save at is a directory, but no filename is specified"
            )
        fig.savefig(Path(path, fname))
    else:
        fig.savefig(path)

def save_pairwise_instability_distribution(
    diffs: np.ndarray, savepath: Optional[Union[str, Path]] = None
):
    if savepath is not None:
        try:
            g = sns.displot(x=diffs)
            g.set_xlabels("Pairwise instable predictions")
            savefig(g, savepath, fname="pairwise_instable.jpg")
            plt.close()
        except Exception as e:  # should only be numpy.core._exceptions.MemoryError
            log.error("Histogram failed (%s)", e)

def normalized_pairwise_instability(
    preds: np.ndarray, accs: np.ndarray, figurepath = None
) -> np.ndarray:
    """Calculate the min-max normalized pairwise instability.
    Args:
        preds (np.ndarray): Predictions of shape (n_models, n_nodes)
        accs (np.ndarray): Model accuracies of shape (n_models)
        figurepath (Optional[Path]): Path to save a plot of the distribution to. Defaults to None.
    """
    assert (
        preds.shape[0] == accs.shape[0]
    ), f"preds and accs must have same size of first dimension: {preds.shape[0]=}, {accs.shape[0]=}"

    diffs = []
    for i, j in itertools.combinations(np.arange(preds.shape[0]), 2):
        x, y = preds[i], preds[j]
        diff = np.not_equal(x, y).sum() / len(x)

        acc1, acc2 = accs[i], accs[j]
        lower = min_pi(acc1, acc2)
        upper = max_pi(acc1, acc2)
        diff = (diff - lower) / (upper - lower)

        diffs.append(diff)
    diffs = np.asarray(diffs)

    if figurepath is not None and figurepath.is_dir():
        figurepath = Path(figurepath, "rel_pi.jpg")

    save_pairwise_instability_distribution(
        diffs, savepath=figurepath
    )
    return diffs

def get_test_accuracies(model_directory):
    objects = []
    with (open(file_path+model_directory+"/"+model_directory+".pkl", "rb")) as openfile:
        while True:
            try:
                objects.append(pickle.load(openfile))
            except EOFError:
                break
    logs = objects[-30:]
    test_accuracies = []
    for log in logs:
      test_accuracies.append((str(log[0]), log[1].item()))
    return sorted(test_accuracies)

In [85]:
from torch_geometric.nn import GIN

class GINWrapper(GIN):

  def __init__(self, in_channels: int, hidden_channels: int, num_layers: int, out_channels: int):
    # Create the model to extract the node embeddings then pass these through a linear layer for classification
    super().__init__(in_channels, hidden_channels, num_layers)
    self.out_channels = out_channels
    self.final_layer = nn.Linear(hidden_channels, out_channels)

  def forward(self, x: torch.Tensor, edge_index: Adj):
    x = super().forward(x, edge_index)
    output = self.final_layer(x)
    return output, x

In [88]:
NUM_NODES_CORA = cs_data.x.shape[0]
# CHANGE these to match you directory naming structure
model_directory = "GIN_Coauthor"

embedding_dir = file_path+model_directory

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model =  GINWrapper(in_channels = cs_data.x.shape[-1], hidden_channels = 128, num_layers=1, out_channels=cs_num_classes).to(device)
cs_data = cs_data.to(device)

In [89]:
from torch_geometric.loader import DataLoader

def run_prediction(embedding_dir, model_raw, dataset, results_dir=None):
    results_dir = None
    # Store results in the embedding directory in a results file if no other location is specified
    if results_dir is None:
      results_dir = embedding_dir + "results/"
    # Create results directory if it does not exist
    if not os.path.exists(results_dir):
        os.mkdir(results_dir) 

    # find all model files
    fnames = sorted([f for f in os.listdir(embedding_dir) if f.endswith("_model.pt")])
    print(fnames)
    predictions = []
    model = model_raw
    for emb in fnames:
        full_path = os.path.join(file_path+model_directory+"/"+emb)
        model.load_state_dict(torch.load(full_path))
        #print("asdf")

        # Now, get the prediction logits for the dataset
        loader = DataLoader(dataset, batch_size=1)
        logits = []

        for data in loader:
            data = data   #.to('cuda')  # use cuda if needed
            model = model #.to('cuda')  # use cuda if needed
            model.eval()
            with torch.no_grad():
                out, x = model(data.x.to(device), data.edge_index.to(device))
                logits.append(out.cpu().numpy())

        predictions.append(logits)
        #print(len(logits))
    predictions = np.stack(predictions).squeeze()
    return predictions
predictions = run_prediction(embedding_dir, model, cs_dataset)


['GIN_Coauthor_1024258131_model.pt', 'GIN_Coauthor_116846604_model.pt', 'GIN_Coauthor_1221215631_model.pt', 'GIN_Coauthor_1517964140_model.pt', 'GIN_Coauthor_170173784_model.pt', 'GIN_Coauthor_1818027900_model.pt', 'GIN_Coauthor_1860537279_model.pt', 'GIN_Coauthor_1863727779_model.pt', 'GIN_Coauthor_2036056847_model.pt', 'GIN_Coauthor_2105922959_model.pt', 'GIN_Coauthor_2135956485_model.pt', 'GIN_Coauthor_2342954646_model.pt', 'GIN_Coauthor_2455059856_model.pt', 'GIN_Coauthor_2739899259_model.pt', 'GIN_Coauthor_2794978777_model.pt', 'GIN_Coauthor_2952735006_model.pt', 'GIN_Coauthor_3300171104_model.pt', 'GIN_Coauthor_3303475786_model.pt', 'GIN_Coauthor_361232447_model.pt', 'GIN_Coauthor_3647665043_model.pt', 'GIN_Coauthor_3692371949_model.pt', 'GIN_Coauthor_3710910636_model.pt', 'GIN_Coauthor_400225693_model.pt', 'GIN_Coauthor_4083009686_model.pt', 'GIN_Coauthor_4193977854_model.pt', 'GIN_Coauthor_516507873_model.pt', 'GIN_Coauthor_572297925_model.pt', 'GIN_Coauthor_806299656_model.pt'

In [90]:
predictions.shape

(30, 18333, 15)

In [91]:
accuracies = get_test_accuracies(model_directory)
acc = [x[1] for x in accuracies]
acc = np.stack(acc)
print(accuracies)
print(acc)

[('1024258131', 88.31698608398438), ('116846604', 88.83441925048828), ('1221215631', 89.5969467163086), ('1517964140', 88.04466247558594), ('170173784', 89.4880142211914), ('1818027900', 89.40631866455078), ('1860537279', 88.34422302246094), ('1863727779', 90.0599136352539), ('2036056847', 89.51525115966797), ('2105922959', 87.52723693847656), ('2135956485', 88.12635803222656), ('2342954646', 88.07189178466797), ('2455059856', 89.89651489257812), ('2739899259', 88.07189178466797), ('2794978777', 88.80718994140625), ('2952735006', 87.09149932861328), ('3300171104', 89.40631866455078), ('3303475786', 88.28975677490234), ('361232447', 88.42591857910156), ('3647665043', 87.88126373291016), ('3692371949', 90.08714294433594), ('3710910636', 89.84204864501953), ('400225693', 88.77995300292969), ('4083009686', 88.99781799316406), ('4193977854', 87.66339111328125), ('516507873', 89.76034545898438), ('572297925', 88.80718994140625), ('806299656', 87.63616180419922), ('880019963', 89.052284240722

In [92]:
normalized_pairwise_instability(predictions, acc)

array([-0.08244244, -0.07742953, -0.08433454, -0.07814037, -0.07867462,
       -0.08571124, -0.07442783, -0.07796247, -0.08137163, -0.08480235,
       -0.08449047, -0.07548367, -0.08449047, -0.08262305, -0.07887648,
       -0.07867462, -0.08573802, -0.08516392, -0.08339888, -0.07425226,
       -0.07583649, -0.08280383, -0.08136093, -0.08215128, -0.07636654,
       -0.08262305, -0.08199536, -0.08100132, -0.08479954, -0.08034968,
       -0.08089223, -0.08106411, -0.08160107, -0.08259748, -0.0773328 ,
       -0.08088532, -0.07794677, -0.08135728, -0.08104723, -0.07839399,
       -0.08104723, -0.08523293, -0.07546632, -0.08160107, -0.08228743,
       -0.08306254, -0.07996207, -0.07715634, -0.07874858, -0.08507789,
       -0.08430096, -0.07872183, -0.07928131, -0.08523293, -0.07856682,
       -0.08393953, -0.08337263, -0.07589266, -0.08403827, -0.08357721,
       -0.07758324, -0.08161378, -0.08419198, -0.07297255, -0.07635371,
       -0.07604633, -0.08268284, -0.07604633, -0.08019601, -0.07

#Accuracy stats

In [37]:
import pickle
import statistics

def get_test_accuracy_stats(model_directory):
    objects = []
    with (open(file_path+model_directory+"/"+model_directory+".pkl", "rb")) as openfile:
        while True:
            try:
                objects.append(pickle.load(openfile))
            except EOFError:
                break
    logs = objects[-30:]
    test_accuracies = []
    for log in logs:
      test_accuracies.append(log[1].item())
    return test_accuracies
def accuracy_stats(model_directory):
    test_accuracies = get_test_accuracy_stats(model_directory)
    print(test_accuracies)

    print("std", statistics.stdev(test_accuracies))
    print("mean", statistics.mean(test_accuracies))
    print("max", max(test_accuracies))
    print("min", min(test_accuracies))

In [41]:
NUM_NODES_CORA = cora_data.x.shape[0]
# CHANGE these to match you directory naming structure
model_directory = "GIN_Coauthor_Layers_5"
accuracy_stats(model_directory)

[70.3431396484375, 49.48257064819336, 53.94880294799805, 54.520694732666016, 59.12309265136719, 37.03703689575195, 42.374725341796875, 37.636165618896484, 43.27341842651367, 43.845314025878906, 50.38125991821289, 57.053375244140625, 40.71350860595703, 50.245094299316406, 54.08496856689453, 56.01852035522461, 56.50871276855469, 66.69389343261719, 37.69063186645508, 48.80174255371094, 58.22439956665039, 46.568626403808594, 33.38779830932617, 55.93682098388672, 65.08714294433594, 59.09585952758789, 60.375816345214844, 49.1285400390625, 64.59695434570312, 66.6394271850586]
std 9.826307514633445
mean 52.29393513997396
max 70.3431396484375
min 33.38779830932617
